In [60]:
from metafid.deriv import Pricing
from metafid.deriv import OptionStrategy
import pandas as pd
import numpy as np
import jdatetime as jdt
import requests
from collections import namedtuple
from itertools import combinations
from operator import add
import re 
import finpy_tse as fpy
import psycopg
import jdatetime as jdt
import numpy as np
import re
from CONFIG import DB_PASS as PASS

In [57]:
bs = Pricing()
now = jdt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
now

'1402-01-02 13:52:31'

In [3]:
"https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/"

'https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/'

In [49]:
drop_cols = ["isin","time","open","close","no","volume","low","high","y_final","eps","base_vol","unknown1","unknown2","sector","day_ul","day_ll", "share_no", "mkt_id", "sell_no", "buy_no"]
ua_list = ["شپنا",
"خودرو",
"فملي",
"شستا",
"وبصادر",
"فولاد",
"خگستر",
"سلام",
"دي",
"سرو",
"خاور",
"وبملت",
"خساپا",
"فخوز",
"فرابورس",
"اهرم",
"حسير",
"خپارس",
"هاي وب",
"چكاپا",
"توسن",
"وتجارت",
"ذوب",
"بساما",
"شبندر",
"سمگا",
"وخاور",
"شتران",
"كاريس",
"ت ثاميد",
"پالايش",
"لبخند",
"تپسي",
"كارا"]

def mw(drop_cols:list=None):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
    }
    r = requests.get(
        "http://www.tsetmc.com/tsev2/data/MarketWatchPlus.aspx", headers=headers
    )
    main_text = r.text
    ob_df = pd.DataFrame((main_text.split("@")[3]).split(";"))
    ob_df = ob_df[0].str.split(",", expand=True)
    ob_df.columns = [
        "web_id",
        "ob_depth",
        "sell_no",
        "buy_no",
        "buy_price",
        "sell_price",
        "buy_vol",
        "sell_vol",
    ]
    ob_df = ob_df[
        [
            "web_id",
            "ob_depth",
            "sell_no",
            "sell_vol",
            "sell_price",
            "buy_price",
            "buy_vol",
            "buy_no",
        ]
    ]
    ob_df.set_index("web_id", inplace=True)

    mw_df = pd.DataFrame((main_text.split("@")[2]).split(";"))
    mw_df = mw_df[0].str.split(",", expand=True)
    mw_df = mw_df.iloc[:, :23]
    mw_df.columns = [
        "web_id",
        "isin",
        "symbol",
        "name",
        "time",
        "open",
        "final",
        "close",
        "no",
        "volume",
        "value",
        "low",
        "high",
        "y_final",
        "eps",
        "base_vol",
        "unknown1",
        "unknown2",
        "sector",
        "day_ul",
        "day_ll",
        "share_no",
        "mkt_id",
    ]
    mw_df.set_index("web_id", inplace=True)
    df = mw_df.join(ob_df)
    df = df.assign(dt=jdt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    if drop_cols:
        return df.drop(columns = drop_cols, axis=1)
    else:
        return df

In [50]:
def option_data():
    mw_df = mw(drop_cols=drop_cols)

    ua_df = mw_df[(mw_df.symbol.isin(ua_list)) & (mw_df.value.astype(int) > 0) & (mw_df.ob_depth.astype(int)==1)].add_prefix("ua_")
    ua_df.rename(columns={"ua_symbol":"ua"}, inplace=True)
    ua_df.drop_duplicates(inplace=True)
    
    mw_df = mw_df[mw_df.name.str.startswith("اختيار")]
    def expand_option_info(df):
        def clean_date(x):
            date_ =re.findall("[0-9]+", x)
            date_ = "".join(date_)
            if len(date_)==8:
                return f"{date_[:4]}-{date_[4:6]}-{date_[6:8]}"
            elif len(date_)==6:
                return f"14{date_[:2]}-{date_[2:4]}-{date_[4:6]}"
            else:
                print("bad data!")
                return None
        def clean_ua(x):
            return x.replace("اختيارخ ", "").replace('اختيارف ' ,"")  
        
        def day_to_ex(ex_date):
            y, m, d = map(int,(tuple(ex_date.split("-"))))
            return (jdt.date(y, m, d) - jdt.date.today()).days
        

        df[["ua", "strike_price","ex_date"]] = df.name.str.split("-", expand= True)
        df = df.assign(ua = df.ua.map(clean_ua))
        df = df.merge(ua_df, on="ua", how="inner")
        df = df[~df.ex_date.isnull()]
        df = df.assign(ex_date = df.ex_date.map(clean_date))
        df = df.assign(t = df.ex_date.map(day_to_ex))
        return df
    df = expand_option_info(mw_df)
    df = df.apply(pd.to_numeric, errors='ignore')
    return df

In [70]:
def query_df(cols, table):
    with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
        with conn.cursor() as cur:
            cur.execute(f"SELECT {cols} FROM {table}")
            cur.fetchone()
            conn.commit()
            return pd.DataFrame([i[0] for i in cur], columns=re.findall("[a-z A-Z]+", cols))
sigma_df = query_df("(ua,sigma)", "sigma")

In [76]:
df = option_data()
def option_type(x):
    return "call" if x.startswith("ض") else "put"
df = df.assign(type=df.symbol.map(option_type))
df = df.merge(sigma_df, on="ua", how="inner")
df["bs"] = df.apply(lambda x: bs.black_scholes(s_0=x["ua_final"], k=x["strike_price"], t=x["t"],sigma=float(x["sigma"]),type_=x["type"]),axis=1)
call = df[df.type =="call"]
put = df[df.type =="put"]


In [7]:
def married_put(df):
    """
    خرید داراییِ پایه و خریدِ همزمان اختیارِ فروش
    سود-زیان  برابرِ اختلافِ ارزشِ روز دارایی (در صورت کاهش تا حد قیمتِ اعمال) با ارزشِ خرید دارایی منهایِ ارزش خرید اختیارِ فروش است.
    :param df: with (ua, symbol, strike_price, ua_sell_price, sell_price) columns. ua: Underlying Asset
    :return: max_pot_profit: حداکثر سود بالقوه- در شرایطی ایجاد می‌شود که قیمتِ داراییِ پایه در تاریخ سر-رسید از قیمتِ اعمال بالاتر-برابر باشد
            break_even : اگر قیمتِ داراییِ پایه از این قیمت پایین-تر بیاد، معامله ،وارد زیان می‌شود
            pct_max_profit :
            current_profit : بر اساسِ قیمتِ کنونیِ داراییِ پایه محاسبه می‌شود
            pct_current_profit :
    """
    df = df[df.sell_price>0]
    stg = namedtuple("Married-Put", "buy buy_")
    df["buy_buy"] = df.apply(
        lambda x: stg(buy=x["ua"], buy_=x["symbol"]), axis=1
    )
    df = df.assign(max_pot_profit="indefinite")
    df = df.assign(break_even=df.ua_sell_price + df.sell_price)
    df["current_profit"] = df.apply(
        lambda x: max(x["strike_price"], x["ua_sell_price"]) - x["ua_sell_price"] - x["sell_price"], axis=1)
    df = df.assign(pct_current_profit=df.current_profit / df.break_even * 100)
    df = df.assign(pct_profit_per_day=df.pct_current_profit / df.t)

    return df.sort_values(by="pct_profit_per_day", ascending=False)

In [77]:
def bear_call_spread(df):
        """
        فروش اختیار خرید با قیمتِ اعمالِ پایین-تر و خرید اختیارِ خریدِ با قیمتِ اعمالِ بالا-تر و تاریخِ اعمالِ همسان
        بیشینه سود برابر است با تفاوتِ بینِ دو قیمتِ اعمال منهایِ پرمیوم پرداختی
        بیشینه ضرر هم برابر است با پرمیوم پرداختی
         :param df: with (ua, symbol, t, strike_price, ua_final, sell_price, buy_price) columns. ua: Underlying Asset
         :return:max_pot_loss: max_pot_loss
                max_pot_profit: max_pot_profit
                current_profit: current_profit
        """
        stg = namedtuple("Bear-Call-Spread", "sell buy")
        groups = df.groupby(by=["ua", "t"])
        df_ = pd.DataFrame()
        for name, group in groups:
            group.reset_index(inplace=True)
            group = group.sort_values(by=["strike_price"], ascending=False)
            if len(group) > 1:
                combo_symbol = list(stg(sell=s, buy=b) for s, b in combinations(group.symbol, 2))
                combo_strike_price = list(stg(sell=s, buy=b) for s, b in combinations(group.strike_price, 2))
                combo_ob_price = list(stg(sell=s, buy=b) for s, b in combinations(group[["sell_price", "buy_price"]].itertuples(index=False), 2))
                combo_bs = list(stg(sell=s, buy=b) for s, b in combinations(group.bs, 2))
                max_pot_loss = [ps.buy_price - pb.sell_price for ps, pb in combo_ob_price]
                max_pot_profit = list(
                    map(add, [ss - sb for ss, sb in combo_strike_price], max_pot_loss)
                )
                current_profit = []
                for i in range(len(combo_strike_price)):
                    if all(
                        s <= group.ua_final.values[0] for s in combo_strike_price[i]
                    ):
                        current_profit.append(max_pot_profit[i])
                    elif all(
                        s >= group.ua_final.values[0] for s in combo_strike_price[i]
                    ):
                        current_profit.append(max_pot_loss[i])
                    else:
                        current_profit.append(
                            group.ua_final.values[0]
                            - combo_strike_price[i].buy
                            + max_pot_loss[i]
                        )

                df_group = pd.DataFrame(
                    {
                        "symbol": combo_symbol,
                        "strike_price": combo_strike_price,
                        "t": group.t.values[0],
                        "bs": combo_bs,
                        "ua_final": group.ua_final.values[0],
                        "ob_price": combo_ob_price,
                        "max_pot_loss": max_pot_loss,
                        "max_pot_profit": max_pot_profit,
                        "current_profit": current_profit,
                    }
                )
                df_ = pd.concat([df_, df_group])
        return df_

In [78]:
df_bsc = bull_call_spread(call[(call.ob_depth==1) &(call.sell_price>0) &(call.buy_price>0)])

In [79]:
df_bsc.to_excel("bull_call_spread.xlsx")

In [36]:
call.columns

Index(['symbol', 'name', 'value', 'ob_depth', 'sell_vol', 'sell_price',
       'buy_price', 'buy_vol', 'dt', 'ua', 'strike_price', 'ex_date',
       'ua_name', 'ua_value', 'ua_ob_depth', 'ua_sell_vol', 'ua_sell_price',
       'ua_buy_price', 'ua_buy_vol', 'ua_dt', 't'],
      dtype='object')